In [ ]:
from datetime import timedelta

class BullCallSpreadAlgorithm(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2017, 4, 1)
        self.SetEndDate(2017, 6, 30)
        self.SetCash(1000000)
        equity = self.AddEquity("GOOG", Resolution.Minute)
        option = self.AddOption("GOOG", Resolution.Minute)
        self.symbol = option.Symbol
    
        # set our strike/expiry filter for this option chain
        option.SetFilter(-6, 6, timedelta(30), timedelta(60))
        # use the underlying equity GOOG as the benchmark
        self.SetBenchmark(equity.Symbol)
        
    def OnData(self,slice):
        for i in slice.OptionChains:
            chains = i.Value
            if not self.Portfolio.Invested: 
                self.TradeOptions(chains) 
 
    def TradeOptions(self,chains):
        # sorted the optionchain by expiration date and choose the furthest date
        expiry = sorted(chains,key = lambda x: x.Expiry, reverse=True)[0].Expiry
        # filter the call and put contract
        call = [i for i in chains if i.Expiry == expiry and i.Right == OptionRight.Call]
        put = [i for i in chains if i.Expiry == expiry and i.Right == OptionRight.Put]
        
        # sorted the contracts according to their strike prices 
        call_contracts = sorted(call,key = lambda x: x.Strike)    
        if len(call_contracts) == 0: return
        self.call = call_contracts[0]
        for i in put:
            if i.Strike == self.call.Strike:
                self.put = i
        self.Buy(self.call.Symbol, 1)
        self.Buy(self.put.Symbol ,1)
    
    def OnOrderEvent(self, orderEvent):
        self.Log(str(orderEvent))